In [10]:
%pip install pandas openai sentence_transformers pinecone-client kaggle

In [11]:
%pip install opendatasets

In [12]:
import os
import pandas as pd
from openai import OpenAI

In [13]:
import opendatasets as od
import pandas

od.download(
	"https://www.kaggle.com/datasets/ravindrasinghrana/job-description-dataset")


Skipping, found downloaded files in "./job-description-dataset" (use force=True to force download)


In [14]:
import pandas as pd
from openai import OpenAI
client = OpenAI(
)

def get_embedding(text, model="text-embedding-ada-002"):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

# Read the CSV file
episodes = pd.read_csv('./job-description-dataset/job_descriptions.csv')


# Take half of the data
episodes = episodes.sample(frac=0.0002, random_state=1).reset_index(drop=True)

# Features to process
# 'Contact Person', 'Contact',
features = ['Experience', 'Preference', 'Role', 'Job Description', 'skills', 'Responsibilities']
label = 'Job Title'
contact_person = 'Contact Person'
contact = 'Contact'
experience = 'Experience'
preference = 'Preference'
role = 'Role'
job_description = 'Job Description'
skills = 'skills'
responsibilities = 'Responsibilities'

# Convert 'work_experience' to string
# episodes['work_experience'] = episodes['work_experience'].astype(str)

# Strip whitespace from string features
# string_features = ['ssc_board', 'work_experience', 'status']
# episodes[string_features] = episodes[string_features].apply(lambda x: x.str.strip())

# Remove any records with null values
episodes = episodes.dropna(subset=features + [label])

# Combine text columns into a single text field for embedding
episodes['combined_text'] = episodes[features].astype(str).agg(' '.join, axis=1)
from sentence_transformers import SentenceTransformer

# Load a pre-trained model


# Generate embeddings for the combined text
episodes['embeddings'] = episodes['combined_text'].apply(lambda x: get_embedding(x))
import pinecone

# Initialize Pinecone
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(
)

# Create an index if not already created
index_name = 'garudahacks'
# if index_name not in pc.list_indexes():
#     pc.create_index(
#         name="garudahacks",
#         dimension=1536,
#         metric="cosine",
#         spec=ServerlessSpec(
#           cloud="aws",
#           region="us-east-1"
#     ))  # Assuming 384-dimensional embeddings
index = pc.Index(index_name)

# Prepare vectors to upsert
vectors_to_upsert = [
    (str(i), embedding, {
            "Job": label,
            "Contact Person": contact_person,
            "Contact": contact,
            "Experience": experience,
            "Preference": preference,
            "Role": role,
            "Job Description": job_description,
            "skills": skills,
            "Responsibilities": responsibilities,
    })
    for i, (embedding, label, contact_person, contact, experience, preference, role, job_description, skills, responsibilities) in enumerate(zip(episodes['embeddings'], episodes['Job Title'], episodes['Contact Person'], episodes['Contact'], episodes['Experience'], episodes['Preference'], episodes['Role'], episodes['Job Description'], episodes['skills'], episodes['Responsibilities']))
]



In [15]:
# Upsert vectors into Pinecone
index.upsert(vectors=vectors_to_upsert, namespace="findjobs")

{'upserted_count': 323}